# <center>COVID-19 Data Map


In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [2]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

### Convert Data to JSON

In [3]:
covid1= json.loads(data)
covid1

{'Global': {'NewConfirmed': 267087,
  'TotalConfirmed': 22675960,
  'NewDeaths': 6025,
  'TotalDeaths': 793658,
  'NewRecovered': 207646,
  'TotalRecovered': 14540461},
 'Countries': [{'Country': 'Afghanistan',
   'CountryCode': 'AF',
   'Slug': 'afghanistan',
   'NewConfirmed': 257,
   'TotalConfirmed': 37856,
   'NewDeaths': 10,
   'TotalDeaths': 1385,
   'NewRecovered': 515,
   'TotalRecovered': 27681,
   'Date': '2020-08-21T12:22:19Z',
   'Premium': {}},
  {'Country': 'Albania',
   'CountryCode': 'AL',
   'Slug': 'albania',
   'NewConfirmed': 155,
   'TotalConfirmed': 7967,
   'NewDeaths': 4,
   'TotalDeaths': 238,
   'NewRecovered': 58,
   'TotalRecovered': 3986,
   'Date': '2020-08-21T12:22:19Z',
   'Premium': {}},
  {'Country': 'Algeria',
   'CountryCode': 'DZ',
   'Slug': 'algeria',
   'NewConfirmed': 411,
   'TotalConfirmed': 40258,
   'NewDeaths': 9,
   'TotalDeaths': 1411,
   'NewRecovered': 310,
   'TotalRecovered': 28281,
   'Date': '2020-08-21T12:22:19Z',
   'Premium': {}

### Normalize Data 

In [4]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,257,37856,10,1385,515,27681,2020-08-21T12:22:19Z
1,Albania,AL,albania,155,7967,4,238,58,3986,2020-08-21T12:22:19Z
2,Algeria,DZ,algeria,411,40258,9,1411,310,28281,2020-08-21T12:22:19Z
3,Andorra,AD,andorra,0,1024,0,53,0,875,2020-08-21T12:22:19Z
4,Angola,AO,angola,29,2044,1,93,44,742,2020-08-21T12:22:19Z
...,...,...,...,...,...,...,...,...,...,...
181,Viet Nam,VN,vietnam,13,1007,0,25,9,542,2020-08-21T12:22:19Z
182,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-08-21T12:22:19Z
183,Yemen,YE,yemen,7,1899,2,541,3,1058,2020-08-21T12:22:19Z
184,Zambia,ZM,zambia,154,10372,5,274,0,9126,2020-08-21T12:22:19Z


### Convert to Pandas DataFrame

In [5]:
df = pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,257,37856,10,1385,515,27681,2020-08-21T12:22:19Z,{}
1,Albania,AL,albania,155,7967,4,238,58,3986,2020-08-21T12:22:19Z,{}
2,Algeria,DZ,algeria,411,40258,9,1411,310,28281,2020-08-21T12:22:19Z,{}
3,Andorra,AD,andorra,0,1024,0,53,0,875,2020-08-21T12:22:19Z,{}
4,Angola,AO,angola,29,2044,1,93,44,742,2020-08-21T12:22:19Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
181,Viet Nam,VN,vietnam,13,1007,0,25,9,542,2020-08-21T12:22:19Z,{}
182,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-08-21T12:22:19Z,{}
183,Yemen,YE,yemen,7,1899,2,541,3,1058,2020-08-21T12:22:19Z,{}
184,Zambia,ZM,zambia,154,10372,5,274,0,9126,2020-08-21T12:22:19Z,{}


### Drop Unnecessary Columns

In [6]:
covid2 = df.drop(columns =['CountryCode','Slug','Date','Premium'],axis=1) 

### Generate Base Map

In [7]:
m = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
m

### Obtain Geodata

In [8]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

### Generate Choropleth Map Layer

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='COVID-19',
    data=covid2,
    columns=['Country', 'TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

### Generate Circular Markers

In [10]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [11]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [12]:
covid_final= pd.merge(covid2,coordinates,on='Country')

In [13]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)


In [14]:
covid_final.apply(plotDot, axis = 1)

m.fit_bounds(m.get_bounds())

m


### Generate Base Map

In [15]:
m1 = folium.Map(tiles='StamenToner', min_zoom=2)

m1

In [17]:
lat=covid_final['latitude'].astype(float)

In [18]:
lon=covid_final['longitude'].astype(float)

In [16]:
deaths=covid_final['TotalDeaths'].astype(float)

### Generate Heat Map Layer

In [19]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

### Generate Circular Marker

In [20]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)

covid_final.apply(plotDot, axis = 1)

m1.fit_bounds(m1.get_bounds())

m1